In [18]:
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from __future__ import division
import matplotlib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [19]:
nltk.download("stopwords")
from nltk.corpus import stopwords
esw = stopwords.words("english")
from string import punctuation

#remove "num", because "num" has the highest term frequency(45538) at the orinal file, 
#the second most frequent term only has a freq. of 3750
esw = esw + ['abstract', 'ci', 'hr','l','pubmed', 'num'] 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hailianhou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
from collections import defaultdict

In [4]:
####read file

#read train.docs file
train_doc_file = pd.read_csv("train.docs", encoding = 'utf-8', sep='\t', header=None)
train_doc_file.columns = ['id', 'text']

#read train.nontopic-titles.queries file
train_query_file = pd.read_csv("train.nontopic-titles.queries", encoding = 'utf-8', sep='\t', header=None)
train_query_file.columns = ['id', 'text']





### preprocessing

train_doc_file["text"] = train_doc_file['text'].str.replace('/', ' or ')

train_doc_file["text"] = train_doc_file['text'].str.replace('-', ' and ')

train_query_file["text"] = train_query_file['text'].str.replace('/', ' or ')

train_query_file["text"] = train_query_file['text'].str.replace('-', ' and ')


# inout file: train_doc_file["text"]/ train_query_file["text"]

def removeStopwords(file):
    i = 0
    for el in file:
        el = ' '.join([word for word in el.lower().split() if word not in esw])
        file[i] = el
        i += 1
def removePunctuation(file):
    i = 0
    for el in file:
        el = ' '.join([word for word in el.lower().split() if word not in punctuation])
        file[i] = el
        i += 1

def stemming(file):
    ps = PorterStemmer()
    i = 0
    for el in file:
        el = ' '.join([ps.stem(word) for word in el.lower().split() ])
        file[i] = el
        i += 1
        


In [5]:
### remove stopwords
removeStopwords(train_doc_file['text'])
removeStopwords(train_query_file['text'])

### stemming
#stemming(train_doc_file['text'])
#stemming(train_query_file['text'])

### remove punctuation
removePunctuation(train_doc_file['text'])
removePunctuation(train_query_file['text'])

In [6]:
### create DTM

 ## get DTM, weighted by tfidf
def get_DTM_tfidf(file):
    train_count_vect = CountVectorizer() #remove english stopwords
    X_train_counts = train_count_vect.fit_transform(file)
    vocabulary = list(train_count_vect.vocabulary_.keys())
    #print(vocabulary)

    tfidf_transformer = TfidfTransformer()
    
    X_train_tfidf = CountVectorizer(vocabulary = vocabulary)
    X_train_tfidf = X_train_tfidf.fit_transform(file)
    
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_tfidf)
    return X_train_tfidf


## get DTM, weighted by term frequency
def get_DTM_tf(file):
    train_count_vect = CountVectorizer() #remove english stopwords
    X_train_counts = train_count_vect.fit_transform(file)
    vocabulary = list(train_count_vect.vocabulary_.keys())

    X_train_tf = CountVectorizer(vocabulary = vocabulary)
    X_train_tf = X_train_tf.fit_transform(file)
    return X_train_tf


In [7]:
### create tfidf weighted DTM for the train.docs file
train_tfidf = get_DTM_tfidf(train_doc_file.text)
train_tfidf

<3612x23449 sparse matrix of type '<class 'numpy.float64'>'
	with 310355 stored elements in Compressed Sparse Row format>

In [8]:
###Generate the query vector
def get_QueryVector(queryFile, docFile):
    train_count_vect = CountVectorizer() #remove english stopwords
    X_train_counts = train_count_vect.fit_transform(docFile)
    vocabulary = list(train_count_vect.vocabulary_.keys())
    
    if type(queryFile) is str:
        query_vect = []
        query = queryFile
        query = query.split()
        frequency = defaultdict(int)
        for el in vocabulary:
            if el in query:
                frequency[el]+= 1
            else:
                frequency[el] = 0
        query_vect = list(dict(frequency).values() )   
    else:
        query_vect = CountVectorizer(analyzer = "word", vocabulary = vocabulary)
        query_vect = query_vect.fit_transform(queryFile)
    return query_vect

In [9]:
### create query vector matrix for the train.nontopic-titles.queries file 
query_vect = get_QueryVector(train_query_file.text, train_doc_file.text)
query_vect

<1141x23449 sparse matrix of type '<class 'numpy.int64'>'
	with 4192 stored elements in Compressed Sparse Row format>

In [10]:
import math
import random

In [11]:
# input: 1* n dims sparse matrix, or single vector from query_vect/ train_tfidf matrix
def getSquareSum(vector):
    squaresum = 0
    for i in range(vector.shape[1]):
        squaresum += vector[0,i]* vector[0,i]
        
    return squaresum 

In [12]:
# input: 1* n sparse matrix, or single vector from query_vect/ train_tfidf matrix
# return the cosine sim of two vectors

def getCosineSimilarity(query_vector, doc_vector):
    squaresum_query = getSquareSum(query_vector)
    squaresum_doc= getSquareSum(doc_vector)
    sim = np.dot(query_vector, doc_vector.transpose())/(math.sqrt(squaresum_query)*math.sqrt(squaresum_doc))
    return sim
    

In [13]:
#test
getCosineSimilarity(query_vect[1], train_tfidf[19])[0,0]

0.0

In [14]:
### information retrieve 
# return1: the randomly selected docLeaders index list, 
# return2: doc_clustering for each docLeader 

# input1: DTM_tfidf: tfidf weighted Document-term-matrix
# input2: preprocessed document file: train_doc_file 
# input3: docLeaders number (int)
def preclusteringByRandomLeader(DTM_tfidf, doc_File, leaderNumber):
    
    train_tfidf = DTM_tfidf
    documents_id = list(doc_File['id'])
    leaderIndex = []
    doc_clustering = []
    
    for i in range(leaderNumber):
        leaderIndex.append(random.randint(0,train_tfidf.shape[0])) # randomly select doc leaders index
        doc_clustering.append([]) # initiate topic clustering

    for i in range(train_tfidf.shape[0]):
        sims = []
        #if i not in index:
        for el in leaderIndex:
            sims.append(np.dot(train_tfidf[i,], train_tfidf[el,].transpose())[0,0])
            #print(sims)    
            #maxSim = max(sims)
        maxsimindex = sims.index(max(sims))
        doc_clustering[maxsimindex].append(documents_id[i]) 
    
    return leaderIndex,doc_clustering 


In [15]:
### IR algorithm, return query results through the cosine similarity between 

def IRqueryByLeaders(leaderIndex, doc_File, doc_clustering, query_File,queryVector ):    
    ### get the similarity of query with each doc leader
    index = leaderIndex
    doc_clustering = doc_clustering
    query_vector = queryVector
    documents_id = list(doc_File['id'])
    query_id = list(query_File['id'])
    r = []
    
    for q in range(queryVector.shape[0]):
        
        sims_leaders = []
        for el in index:
            sims_leaders.append(np.dot(query_vector[q], train_tfidf[el,].transpose())[0,0])

       
        maxsimindex = sims_leaders.index(max(sims_leaders)) # get the most similarity clustering index
        #print(doc_clustering[maxsimindex])

        sims_docs = []
        #if len(doc_clustering[maxsimindex])> 3:
            #get_sims_docs = []
        for el in doc_clustering[maxsimindex]:
            sims_docs.append(np.dot(query_vector[q], train_tfidf[documents_id.index(el),].transpose())[0,0]) 
            # get the similarty of query&docs in the most similarity clustering index
        
        IR_doc_sims = []
        IR_doc = []
        for i in range(len(doc_clustering[maxsimindex])):
            if sims_docs[i]>0:
                IR_doc_sims.append(sims_docs[i]) #get the non-zero similarity
                IR_doc.append(doc_clustering[maxsimindex][i]) #get the index of the docs with non-zero similarity  


        #IR_doc= []
        #for el in d:
            #IR_doc.append(doc_clustering[maxsimindex][el])


        #print(sims_leaders, sims_docs, d, IR_doc, IR_doc_sims)

        #print("Doc", "Similarity")
        
        
        for j in range(len(IR_doc)):
            
            r.append([str(query_id[q]) ,  str(IR_doc[j]), IR_doc_sims[j]])

    return r
    #t2 = datetime.datetime.now().time()
    
    #print("time:",  t1, t2 )
    #return sims_docs
    
    #sims_docs_normalized = sims_docs/total
           
        
    

In [16]:
# get the leader docs index and the docs clusterings with clustering number =  int(math.sqrt(train_tfidf.shape[0]))
leaderIndex,doc_clustering =  preclusteringByRandomLeader( train_tfidf, train_doc_file, int(math.sqrt(train_tfidf.shape[0])))

In [17]:
# get the leader docs index and the docs clusterings with clustering number =  int(math.sqrt(train_tfidf.shape[0]))
leaderIndex_10,doc_clustering_10 =  preclusteringByRandomLeader( train_tfidf, train_doc_file, 10)

In [21]:
import datetime

In [22]:
################ IRByRandomLeaderPreClustering result1, need to get the performance result #########################
# without being stemmed
# test the IR on the whole query file, #docLeaders = squareroot of total docs number, "-" converted to "and"
t1 =  datetime.datetime.now()
IR_results = IRqueryByLeaders(leaderIndex, train_doc_file, doc_clustering, train_query_file, query_vect )

t2 =  datetime.datetime.now()
t = t2-t1
print("running time:", t )

# save result
df = pd.DataFrame(IR_results, columns = ['QUERY_ID', 'DOC_ID', 'sim_results'])
df.to_csv('IRByRandomLeaderPreClustering result1.txt', header=None, index=None, sep=' ', mode='a')
df

running time: 0:01:25.360460


,QUERY_ID,DOC_ID,sim_results
0,PLAIN-10,MED-1003,0.192972
1,PLAIN-10,MED-1143,0.083021
2,PLAIN-10,MED-1153,0.346493
3,PLAIN-10,MED-1166,0.348091
4,PLAIN-10,MED-1484,0.077504
5,PLAIN-10,MED-1760,0.428321
6,PLAIN-10,MED-1804,0.194483
7,PLAIN-10,MED-1993,0.178715
8,PLAIN-10,MED-1996,0.281905
9,PLAIN-10,MED-2054,0.486856


In [23]:
################ IRByRandomLeaderPreClustering result2, need to get the performance result #########################
# without being stemmed
# test the IR on the whole query file, #docLeaders = 10, "-" converted to "and"
t1 =  datetime.datetime.now()
IR_results = IRqueryByLeaders(leaderIndex_10, train_doc_file, doc_clustering_10, train_query_file, query_vect)
t2 =  datetime.datetime.now()
t = t2-t1
print("running time:", t )

# save result
df = pd.DataFrame(IR_results, columns = ['QUERY_ID', 'DOC_ID', 'sim_results'])
df.to_csv('IRByRandomLeaderPreClustering result2.txt', header=None, index=None, sep=' ', mode='a')
df

running time: 0:04:51.490930


,QUERY_ID,DOC_ID,sim_results
0,PLAIN-10,MED-901,0.142555
1,PLAIN-10,MED-1166,0.348091
2,PLAIN-10,MED-1171,0.275017
3,PLAIN-10,MED-1412,0.115494
4,PLAIN-10,MED-1533,0.252962
5,PLAIN-10,MED-1538,0.121870
6,PLAIN-10,MED-1599,0.081301
7,PLAIN-10,MED-1729,0.330730
8,PLAIN-10,MED-1760,0.428321
9,PLAIN-10,MED-1763,0.268002
